In [1]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
import sys
sys.path.append("../")
from functools import reduce
import operator
import geopandas as gpd
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

In [2]:

from src.geocode import geocode
from src.getFromDict import getFromDict
from src.create_marker import *

Url de datos Madrid del catálogo de las escuelas infantiles

In [3]:
url = 'https://datos.madrid.es/egob/catalogo/202318-0-escuelas-infantiles.json'

Hacemos el .get para obtener los datos

In [4]:
resp = requests.get (url = url)


Obtenemos el .json

In [5]:
data = json.loads(resp.text)
data

{'@context': {'c': 'http://www.w3.org/2002/12/cal#',
  'dcterms': 'http://purl.org/dc/terms/',
  'geo': 'http://www.w3.org/2003/01/geo/wgs84_pos#',
  'loc': 'http://purl.org/ctic/infraestructuras/localizacion#',
  'org': 'http://purl.org/ctic/infraestructuras/organizacion#',
  'vcard': 'http://www.w3.org/2006/vcard/ns#',
  'schema': 'https://schema.org/',
  'title': 'vcard:fn',
  'id': 'dcterms:identifier',
  'relation': 'dcterms:relation',
  'references': 'dcterms:references',
  'address': 'vcard:adr',
  'area': 'loc:barrio',
  'district': 'loc:distrito',
  'locality': 'vcard:locality',
  'postal-code': 'vcard:postal-code',
  'street-address': 'vcard:street-address',
  'location': 'vcard:geo',
  'latitude': 'geo:lat',
  'longitude': 'geo:long',
  'organization': 'vcard:org',
  'organization-desc': 'dcterms:description',
  'accesibility': 'org:accesibilidad',
  'services': 'org:servicios',
  'schedule': 'org:horario',
  'organization-name': 'vcard:organization-name',
  'description': '

Comprobamos las keys, para ver donde estan los datos que necesitamos, que son nombre del cole, latitud y longitud

In [6]:
data.keys()

dict_keys(['@context', '@graph'])

Comprobamos que los datos que necesitamos estan el la key = '@graph'

In [7]:
data_graph = data.get('@graph')

In [8]:
data_graph[0]

{'@id': 'https://datos.madrid.es/egob/catalogo/tipo/entidadesyorganismos/58241-escuela-infantil-municipal-agua-dulce.json',
 '@type': 'https://datos.madrid.es/egob/kos/entidadesYorganismos/CentrosEducacion/EscuelasInfantilesPublicas',
 'id': '58241',
 'title': 'Escuela infantil municipal Agua Dulce',
 'relation': 'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=bfa48ab43d6bb410VgnVCM100000171f5a0aRCRD&vgnextoid=2b0e8ed67151c010VgnVCM2000000c205a0aRCRD',
 'address': {'district': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Tetuan'},
  'area': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Tetuan/Barrio/BellasVistas'},
  'locality': 'MADRID',
  'postal-code': '28039',
  'street-address': 'CALLE LEÑEROS 25'},
 'location': {'latitude': 40.45406088024461, 'longitude': -3.7093713617566357},
 'organization': {'organization-desc': 'ENSEÑANZA: Educación Infantil (Primer Ciclo)BUS: 44 , 128',
  'accesibility': '0'

Hacemos una pureba con el primer  elemento y comprobamos que los datos que necesitamos son title y location

In [9]:

for k,v in data_graph[0].items():
    print(k,v,"\n")


@id https://datos.madrid.es/egob/catalogo/tipo/entidadesyorganismos/58241-escuela-infantil-municipal-agua-dulce.json 

@type https://datos.madrid.es/egob/kos/entidadesYorganismos/CentrosEducacion/EscuelasInfantilesPublicas 

id 58241 

title Escuela infantil municipal Agua Dulce 

relation http://www.madrid.es/sites/v/index.jsp?vgnextchannel=bfa48ab43d6bb410VgnVCM100000171f5a0aRCRD&vgnextoid=2b0e8ed67151c010VgnVCM2000000c205a0aRCRD 

address {'district': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Tetuan'}, 'area': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Tetuan/Barrio/BellasVistas'}, 'locality': 'MADRID', 'postal-code': '28039', 'street-address': 'CALLE LEÑEROS 25'} 

location {'latitude': 40.45406088024461, 'longitude': -3.7093713617566357} 

organization {'organization-desc': 'ENSEÑANZA: Educación Infantil (Primer Ciclo)BUS: 44 , 128', 'accesibility': '0', 'schedule': '', 'services': 'Horario amplia

In [10]:
map_name = ["title"]
lat = ["location","latitude"]
long = ["location","longitude"]

In [11]:
list_school = []
for dict_elem in data_graph:
    list_test = {}
    list_test["name"] = getFromDict(dict_elem,map_name)
    list_test["latitud"] = getFromDict(dict_elem,lat)
    list_test["longitud"] = getFromDict(dict_elem,long)
    list_school.append(list_test)

In [12]:
len(list_school)

59

Obtenemos el dataframe con los datos seleccionados

In [13]:
dataframefinal = pd.DataFrame(list_school)
dataframefinal.head()

,name,latitud,longitud
0,Escuela infantil municipal Agua Dulce,40.454061,-3.709371
1,Escuela infantil municipal Altair,40.378132,-3.768701
2,Escuela infantil municipal Amanecer,40.401990,-3.601044
3,Escuela infantil municipal Ana de Austria,40.495522,-3.655737
4,Escuela infantil municipal Campanilla,40.452700,-3.701998


In [22]:
school = dataframefinal.to_csv("./data/school.csv")

Le añadimos una columna geomtery para poder hacer los mapas

In [14]:
gdf = gpd.GeoDataFrame(dataframefinal, geometry=gpd.points_from_xy(dataframefinal.longitud, dataframefinal.latitud))
print(f'Tipo: {type(gdf)}')
gdf.head()

Tipo: <class 'geopandas.geodataframe.GeoDataFrame'>


,name,latitud,longitud,geometry
0,Escuela infantil municipal Agua Dulce,40.454061,-3.709371,POINT (-3.70937 40.45406)
1,Escuela infantil municipal Altair,40.378132,-3.768701,POINT (-3.76870 40.37813)
2,Escuela infantil municipal Amanecer,40.401990,-3.601044,POINT (-3.60104 40.40199)
3,Escuela infantil municipal Ana de Austria,40.495522,-3.655737,POINT (-3.65574 40.49552)
4,Escuela infantil municipal Campanilla,40.452700,-3.701998,POINT (-3.70200 40.45270)


In [15]:
address = "Madrid"
madrid = geocode(address)

In [16]:
madrid.get("coordinates")
print(madrid.get("coordinates"))
initial_long = madrid.get("coordinates")[0]
initial_lat = madrid.get("coordinates")[1]
initial_lat

[-3.6793, 40.42955]


40.42955

In [17]:
map_1 = folium.Map(location = [initial_lat,initial_long], zoom_start = 15)
map_1

In [19]:
icon = "graduation-cap"
create_marker_school(gdf,map_1)

In [20]:
map_1